# A tensorflow model for Dictionary Filter

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf

2024-07-14 11:57:31.558859: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-14 11:57:31.583233: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-14 11:57:31.959168: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
from NMSE import NMSE
from BatchGaussianDF import BatchGaussianDF as DictionaryFilter

## Test on Movie Lens 100k dataset

In [3]:
# Read the dataset
dataset = pd.read_csv('dataset.csv')

# Display the dataset
dataset.head()

Movie Index  User Index  Rating
0          356         137     4.0
1          708          91     2.0
2          411         300     4.0
3           55          59     4.0
4          894         196     3.0

In [5]:
from sklearn.model_selection import train_test_split
from AdvancedModelSelection import user_based_train_test_split

# Split the dataset into train and test data
train_val_data, test_data = user_based_train_test_split(dataset, test_size=0.2, random_state=42)

# Split the train and validation data
train_data, val_data = train_test_split(train_val_data, test_size=0.1, random_state=42)

# Print the shapes of train and test data
print("Train data shape:", train_data.shape)
print("Validation data shape:", val_data.shape)
print("Test data shape:", test_data.shape)

Train data shape: (72000, 3)
Validation data shape: (8000, 3)
Test data shape: (20000, 3)


In [6]:
NUM_MOVIES = dataset['Movie Index'].max() + 1
NUM_USERS = dataset['User Index'].max() + 1
NUM_FACTORS = 2
BATCH_SIZE = 32
SIGMA = 0.2
GAMMA = 0.01
INIT_SCALE = dataset['Rating'].mean() / NUM_FACTORS

In [7]:
# Convert train data to sparse tensor
train_sparse_tensor = tf.sparse.SparseTensor(
    indices=train_data[['Movie Index', 'User Index']].values,
    values=train_data['Rating'].values,
    dense_shape=[NUM_MOVIES, NUM_USERS]
)
train_sparse_tensor = tf.sparse.reorder(train_sparse_tensor)

# Convert validation data to sparse tensor
val_sparse_tensor = tf.sparse.SparseTensor(
    indices=val_data[['Movie Index', 'User Index']].values,
    values=val_data['Rating'].values,
    dense_shape=[NUM_MOVIES, NUM_USERS]
)
val_sparse_tensor = tf.sparse.reorder(val_sparse_tensor)

# Convert test data to sparse tensor
test_sparse_tensor = tf.sparse.SparseTensor(
    indices=test_data[['Movie Index', 'User Index']].values,
    values=test_data['Rating'].values,
    dense_shape=[NUM_MOVIES, NUM_USERS]
)
test_sparse_tensor = tf.sparse.reorder(test_sparse_tensor)

# Create dataset
def data_generator():
    train_slices = tf.sparse.split(sp_input=train_sparse_tensor, num_split=NUM_USERS // BATCH_SIZE, axis=1)
    val_slices = tf.sparse.split(sp_input=val_sparse_tensor, num_split=NUM_USERS // BATCH_SIZE, axis=1)
    test_slices = tf.sparse.split(sp_input=test_sparse_tensor, num_split=NUM_USERS // BATCH_SIZE, axis=1)
    for i in range(NUM_USERS // BATCH_SIZE):
        yield (tf.sparse.to_dense(train_slices[i]), tf.sparse.to_dense(val_slices[i]), tf.sparse.to_dense(test_slices[i]))

dataset = tf.data.Dataset.from_generator(
    data_generator, 
    output_signature=(
        tf.TensorSpec(shape=[NUM_MOVIES, None], dtype=tf.float32),
        tf.TensorSpec(shape=[NUM_MOVIES, None], dtype=tf.float32),
        tf.TensorSpec(shape=[NUM_MOVIES, None], dtype=tf.float32)
    )
)

2024-07-14 11:57:56.817653: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-07-14 11:57:56.834094: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-07-14 11:57:56.834130: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-07-14 11:57:56.835681: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-07-14 11:57:56.835714: I external/local_xla/xla/stream_executor

In [8]:
# Create the model
model = DictionaryFilter(NUM_MOVIES, NUM_FACTORS, INIT_SCALE, SIGMA, GAMMA)
nmse_train = NMSE()
nmse_val = NMSE()
nmse_test = NMSE()

# Train the model
for epoch in range(10):
    print("Epoch:", epoch)
    for train_batch, val_batch, test_batch in dataset:
        x = model(train_batch)
        
        # Perform some gradient descent approximations of X
        if epoch > 5:
            for _ in range(5):
                x = model(train_batch, x)

        nmse_train.update_state(train_batch, tf.matmul(model.C, x), tf.cast(tf.not_equal(train_batch, 0.0), tf.float32))
        nmse_val.update_state(val_batch, tf.matmul(model.C, x), tf.cast(tf.not_equal(val_batch, 0.0), tf.float32))
        nmse_test.update_state(test_batch, tf.matmul(model.C, x), tf.cast(tf.not_equal(test_batch, 0.0), tf.float32))
    
    print("Train NMSE:", nmse_train.result().numpy())
    print("Val NMSE:", nmse_val.result().numpy())
    print("Test NMSE:", nmse_test.result().numpy())

Epoch: 0


2024-07-14 11:57:57.857871: I tensorflow/core/util/cuda_solvers.cc:178] Creating GpuSolver handles for stream 0x55a84727d070
2024-07-14 11:57:59.493349: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Train NMSE: 0.06969058
Val NMSE: 0.07305633
Test NMSE: 0.07508419
Epoch: 1


2024-07-14 11:58:00.541776: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Train NMSE: 0.06896528
Val NMSE: 0.07274156
Test NMSE: 0.074628204
Epoch: 2


...

2024-07-14 11:58:08.360201: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Train NMSE: 113.555275
Val NMSE: 747.45496
Test NMSE: 552.27216
Epoch: 8


2024-07-14 11:58:10.044636: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Train NMSE: 101.88278
Val NMSE: 665.1302
Test NMSE: 492.03528
Epoch: 9
Train NMSE: 91.77721
Val NMSE: 598.8282
Test NMSE: 443.04068


2024-07-14 11:58:11.717994: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
